In [2]:
# Imports

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from skorch import NeuralNetClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix
    , accuracy_score, ConfusionMatrixDisplay
    )
from torchmetrics import Accuracy, ConfusionMatrix
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from timeit import default_timer as timer
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

def parse_xml(xml_file, img_folder):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    data = []
    
    for image in root.findall('image'):
        img_name = image.get('name')
        label = image.get('label')
        data.append((img_name, label))
    
    # Create a DataFrame
    df = pd.DataFrame(data, columns=['image_name', 'label'])
    df['image_path'] = df['image_name'].apply(lambda x: os.path.join(img_folder, x))
    
    return df

# Set paths
img_folder = '../../data/svt1/img/'
train_xml = '../../data/svt1/train.xml'
test_xml = '../../data/svt1/test.xml'

# Parse the XML files
train_data = parse_xml(train_xml, img_folder)
test_data = parse_xml(test_xml, img_folder)

train_data.head()
test_data.head()

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'NoneType'

In [5]:
def print_xml_structure(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    for image in root.findall('image'):
        img_name = image.get('name')
        label = image.get('label')
        print(f"Image Name: {img_name}, Label: {label}")

# Check structure for train.xml
print_xml_structure(train_xml)

Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name: None, Label: None
Image Name

In [6]:
def parse_xml(xml_file, img_folder):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    data = []
    
    for image in root.findall('image'):
        img_name = image.get('name')
        label = image.get('label')
        
        if img_name is not None and label is not None:
            data.append((img_name, label))
    
    # Create a DataFrame
    df = pd.DataFrame(data, columns=['image_name', 'label'])
    df['image_path'] = df['image_name'].apply(lambda x: os.path.join(img_folder, x))
    
    return df

In [7]:
train_data = parse_xml(train_xml, img_folder)
train_data.isnull().sum()  # Check for any null values
train_data.head()

image_name    0
label         0
image_path    0
dtype: int64

,image_name,label,image_path


In [8]:
def load_data(dataframe):
    images = []
    labels = []
    for _, row in dataframe.iterrows():
        img_path = row['image_path']
        if os.path.exists(img_path):
            img = plt.imread(img_path)
            images.append(img)
            labels.append(row['label'])  # Convert labels to numerical if needed
        else:
            print(f"Missing image: {img_path}")
    return images, labels

In [9]:
def print_xml_content(xml_file):
    with open(xml_file, 'r') as file:
        content = file.read()
    print(content)

# Print the content of train.xml
print_xml_content(train_xml)

<?xml version="1.0" encoding="utf-8"?>
<tagset>
   <image>
      <imageName>img/14_03.jpg</imageName>
      <address>341 Southwest 10th Avenue Portland OR</address>
      <lex>LIVING,ROOM,THEATERS,KENNY,ZUKE,DELICATESSEN,CLYDE,COMMON,ACE,HOTEL,PORTLAND,ROSE,CITY,BOOKS,STUMPTOWN,COFFEE,ROASTERS,RED,CAP,GARAGE,FISH,GROTTO,SEAFOOD,RESTAURANT,AURA,RESTAURANT,LOUNGE,ROCCO,PIZZA,PASTA,BUFFALO,EXCHANGE,MARK,SPENCER,LIGHT,FEZ,BALLROOM,READING,FRENZY,ROXY,SCANDALS,MARTINOTTI,CAFE,DELI,CROWSENBERG,HALF</lex>
      <Resolution x="1280" y="880"/>
      <taggedRectangles>
         <taggedRectangle height="75" width="236" x="375" y="253">
            <tag>LIVING</tag>
         </taggedRectangle>
         <taggedRectangle height="76" width="175" x="639" y="272">
            <tag>ROOM</tag>
         </taggedRectangle>
         <taggedRectangle height="87" width="281" x="839" y="283">
            <tag>THEATERS</tag>
         </taggedRectangle>
      </taggedRectangles>
   </image>
   <image>
      <ima

In [10]:
def parse_xml(xml_file, img_folder):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    data = []
    
    # Adjust this part based on the actual XML structure
    for image in root.findall('.//image'):  # Check the correct path to 'image' elements
        img_name = image.get('name')  # Change if the attribute name differs
        label = image.get('label')     # Change if the attribute name differs
        
        # Only append if both values are available
        if img_name and label:
            data.append((img_name, label))
    
    # Create a DataFrame
    df = pd.DataFrame(data, columns=['image_name', 'label'])
    df['image_path'] = df['image_name'].apply(lambda x: os.path.join(img_folder, x))
    
    return df

In [11]:
# Parse the XML files again
train_data = parse_xml(train_xml, img_folder)
print(train_data.isnull().sum())  # Check for any null values
print(train_data.head())           # Display the first few rows

image_name    0
label         0
image_path    0
dtype: int64
Empty DataFrame
Columns: [image_name, label, image_path]
Index: []
